#Exercice 1


## Première partie

In [ ]:
import hashlib

def find_string_with_sha256_ending_zeros(names, target_zeros=5):
    """
    Trouve une chaîne de caractères contenant les noms donnés qui, hashée avec SHA-256,
    se termine par un nombre spécifié de zéros en hexadécimal.
    """
    nonce = 0
    while True:
        test_string = f'{names}{nonce}'.encode('utf-8')
        hash_result = hashlib.sha256(test_string).hexdigest()
        if hash_result.endswith('0' * target_zeros):
            print(f"Chaîne trouvée: {test_string.decode('utf-8')} -> {hash_result}")
            break
        nonce += 1

# Utilisation de la fonction avec un exemple
names = 'RaphaelVeron'
target_zeros = 5
find_string_with_sha256_ending_zeros(names, target_zeros)

Chaîne trouvée: RaphaelVeron56057 -> be3562e5164e07265a8c279dd98f185e026618fb3535ee58c895b92b0f600000


## Deuxième partie.

In [ ]:
import time
import hashlib

def time_to_find_zeros(names, target_zeros=5, trials=5):
    """
    Mesure le temps moyen nécessaire pour obtenir un hash se terminant par n zéros,
    puis pour (n+1) zéros, et calcule le rapport des deux temps.
    """
    def find_zeros(target_zeros):
        nonce = 0
        start_time = time.time()
        while True:
            test_string = f'{names}{nonce}'.encode('utf-8')
            hash_result = hashlib.sha256(test_string).hexdigest()
            if hash_result.endswith('0' * target_zeros):
                return time.time() - start_time
            nonce += 1

    time_n = sum(find_zeros(target_zeros) for _ in range(trials)) / trials
    time_n_plus_1 = sum(find_zeros(target_zeros + 1) for _ in range(trials)) / trials

    print(f"Temps moyen pour {target_zeros} zéros: {time_n} secondes")
    print(f"Temps moyen pour {target_zeros + 1} zéros: {time_n_plus_1} secondes")
    print(f"Rapport du temps pour (n+1) / n: {time_n_plus_1 / time_n}")

# Utilisation de la fonction avec un exemple
names = 'RaphaelVeron'
target_zeros = 5
trials = 5
time_to_find_zeros(names, target_zeros, trials)


Temps moyen pour 5 zéros: 0.15275588035583496 secondes
Temps moyen pour 6 zéros: 14.430744743347168 secondes
Rapport du temps pour (n+1) / n: 94.46932392868726


# Exercice 2

## Étape 1: Chiffrement OpenSSL AES-256 CTR

In [ ]:
openssl enc -aes-256-ctr -pbkdf2 -salt -in monTexte.txt -out monTexte.enc -k monMotDePasse


## Étape 2: Chiffrement ElGamal de 𝑁

In [ ]:
import random

def pgcd(a, b):
    while b:
        a, b = b, a % b
    return a

def trouver_k_coprime(p):
    k = random.randint(2, p - 2)
    while pgcd(k, p - 1) != 1:
        k = random.randint(2, p - 2)
    return k

def generer_cles_elgamal(p, g):
    x = random.randint(2, p - 2)  # Génération de la clé privée
    A = pow(g, x, p)  # Calcul de la clé publique A = g^x mod p
    return x, A  # Retourne la clé privée et la clé publique

def chiffrement_elgamal(N, p, g):
    x, A = generer_cles_elgamal(p, g)
    k = trouver_k_coprime(p)
    B = pow(g, k, p)
    s = pow(A, k, p)
    C = (N * s) % p
    print("Clé privée (x):", x)
    print("Clé publique (A):", A)
    print("B:", B)
    print("C:", C)


test

In [ ]:
p = 7946851324679854613245823
g = 5

N = 354265170000756431  # Exemple basé sur votre description
chiffrement_elgamal(N, p, g)

Clé privée (x): 3334761106309143621309121
Clé publique (A): 3644597538307122694882225
B: 4372155850158119335074090
C: 5126045165226968199505997


1. La fonction trouver_k_coprime génère un k aléatoire qui est coprime avec p−1, c'est-à-dire que k et p−1 ont un PGCD de 1. Cela est nécessaire pour que k soit valide dans le chiffrement ElGamal.

2. pow(base, exp, mod) est utilisé pour le calcul modulaire efficace de base^exp mod mod.

3. Ce code utilise une approche très simplifiée et n'intègre pas les aspects de sécurité importants tels que le choix sécurisé de k. Dans une implémentation réelle de cryptographie, il est crucial d'utiliser des méthodes cryptographiquement sécurisées pour générer k et traiter les données.

## Étape 3: Déchiffrement ElGamal de 𝑁

In [ ]:
def dechiffrement_elgamal(B, C, p, x):
    """
    Déchiffre un message C chiffré avec ElGamal.

    :param B: Partie B du message chiffré.
    :param C: Partie C du message chiffré (le message réellement chiffré).
    :param p: Le nombre premier utilisé pour le chiffrement ElGamal.
    :param x: La clé privée correspondant à la clé publique A utilisée pour le chiffrement.
    :return: Le message déchiffré N.
    """
    s_inv = pow(B, p-1-x, p)  # Calcule l'inverse de s = B^x mod p
    N = (C * s_inv) % p  # Calcule N = C * s^-1 mod p pour obtenir le message déchiffré
    return N



Message déchiffré: 354265170000756431


TEST :

In [ ]:

# Exemple d'utilisation:
p = 7946851324679854613245823
# La clé privée x doit être connue, elle est nécessaire pour le déchiffrement
x = 4574622722378040049760828
B = 5306592376829635731063413  # Utilisez les valeurs B et C obtenues lors du chiffrement
C = 1337481536066283451780629

# Déchiffrement du message
N_dechiffre = dechiffrement_elgamal(B, C, p, x)
print("Message déchiffré:", N_dechiffre)

Message déchiffré: 354265170000756431


# Nous pouvons proposer une méthode de chiffrement plus compléte

## Chiffrement

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import os

def aes_ctr_encryption(input_text, password, salt,iv):
    """
    Chiffre un texte en utilisant AES-256 en mode CTR avec PBKDF2 pour la dérivation de clé.
    Retourne le texte chiffré et le vecteur d'initialisation utilisé.
    """
    # Génération de la clé à partir du mot de passe
    backend = default_backend()
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=backend
    )
    key = kdf.derive(password.encode())

    # Initialisation du chiffreur AES en mode CTR et sauvegarde de l'IV

    cipher = Cipher(algorithms.AES(key), modes.CTR(iv), backend=backend)
    encryptor = cipher.encryptor()

    # Chiffrement du texte
    ct = encryptor.update(input_text.encode()) + encryptor.finalize()
    print(f"Texte chiffré (hex): {ct.hex()}")
    print(f"Vecteur d'initialisation (hex): {iv.hex()}")
    return ct, iv  # Retourne à la fois le texte chiffré et l'IV utilisé




TEST


In [ ]:
# Exemple d'utilisation de la fonction mise à jour
iv =  b'\x6a\x49\xd5\xbe\x1d\xec\xdf\xe4\x34\x1d\xdf\x5c\xfa\xb2\xe3\xd9'
input_text = "Ceci est un exemple de texte à chiffrer."
password = "123456789"
salt = b'bonjour'
encrypted_text, iv_used_for_encryption = aes_ctr_encryption(input_text, password, salt,iv)


Texte chiffré (hex): 65dfbb1171b2ae3bdbd83452b7469be2a735c0aa839d62794887e180b585ef0976a2b40b9cac8e1686
Vecteur d'initialisation (hex): 6a49d5be1decdfe4341ddf5cfab2e3d9


## Déchiffrement

In [ ]:
def aes_ctr_decryption(encrypted_text_hex, password, salt, iv):
    """
    Déchiffre un texte chiffré en utilisant AES-256 en mode CTR avec PBKDF2 pour la dérivation de clé.
    Utilise le même mot de passe, sel (salt), et vecteur d'initialisation (IV) que pour le chiffrement.

    :param encrypted_text_hex: Le texte chiffré à déchiffrer, sous forme de chaîne hexadécimale.
    :param password: Le mot de passe utilisé pour générer la clé de chiffrement.
    :param salt: Le sel utilisé avec le mot de passe pour générer la clé de chiffrement.
    :param iv: Le vecteur d'initialisation utilisé pour le chiffrement.
    :return: Le texte déchiffré.
    """
    # Conversion du texte chiffré hexadécimal en données binaires
    encrypted_text = bytes.fromhex(encrypted_text_hex)

    backend = default_backend()
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=backend
    )
    key = kdf.derive(password.encode())

    cipher = Cipher(algorithms.AES(key), modes.CTR(iv), backend=backend)
    decryptor = cipher.decryptor()

    decrypted_text = decryptor.update(encrypted_text) + decryptor.finalize()
    return decrypted_text.decode('utf-8', errors='ignore')  # Utilise errors='ignore' pour ignorer les erreurs de décodage



test

In [ ]:
# Utilisation de la fonction de déchiffrement
password = "123456789"
salt = b'bonjour'  # Utilisez le même sel que celui utilisé pour le chiffrement
iv = b'\x6a\x49\xd5\xbe\x1d\xec\xdf\xe4\x34\x1d\xdf\x5c\xfa\xb2\xe3\xd9'  # Assurez-vous d'utiliser l'IV correct sous forme de bytes
encrypted_text_hex = "65dfbb1171b2ae3bdbd83452b7469be2a735c0aa839d62794887e180b585ef0976a2b40b9cac8e1686"
decrypted_text = aes_ctr_decryption(encrypted_text_hex, password, salt, iv)
print(f"Texte déchiffré: {decrypted_text}")


Texte déchiffré: Ceci est un exemple de texte à chiffrer.


# Exercice 3.

## Première partie: Démonstration de la signature ElGamal

La signature ElGamal est un système de signature numérique qui est basé sur le problème du logarithme discret.
Pour générer une signature sur un message, on procède comme suit:
1. Choix d'un nombre premier p et d'un générateur g d'un groupe cyclique de Z/pZ.
2. Choix d'une clé privée x par l'émetteur, et calcul de la clé publique y = g^x mod p.
3. Pour signer un message m, l'émetteur choisit un k aléatoire coprime avec p-1, et calcule r = g^k mod p.
4. L'émetteur calcule ensuite s tel que m = x*r + k*s mod (p-1).
5. La signature du message m est le couple (r, s).

Pour vérifier la signature, le récepteur utilise la clé publique y pour s'assurer que g^m mod p = y^r * r^s mod p.
Si l'égalité est vérifiée, alors la signature est valide.

## Deuxième partie: Exemple d'application

Un exemple d'application de la cryptographie dans la vie réelle est l'utilisation de HTTPS pour sécuriser les communications sur Internet.
HTTPS utilise TLS (Transport Layer Security) qui emploie des algorithmes de cryptographie asymétrique pour l'échange de clés (comme RSA ou Diffie-Hellman) et de cryptographie symétrique pour le chiffrement des données (comme AES).
Cela permet de s'assurer que les données échangées entre le navigateur de l'utilisateur et le serveur web sont chiffrées, protégeant ainsi contre les écoutes indiscrètes et les attaques de type man-in-the-middle.


## MAIN FONCTION : On définis un main pour appeler les fonctions que l'on souhaites et testé ce que l'on veut


In [ ]:
def main():
    p = 7946851324679854613245823
    g = 5


    salt = b'bonjour'
    iv = os.urandom(16)

    while True:
        print("\nChoisissez une fonction:")
        print("1: find_string_with_sha256_ending_zeros")
        print("2: time_to_find_zeros")
        print("3: aes_ctr_encryption")
        print("4: Déchiffrement AES CTR")
        print("5: Chiffrement ElGamal de 𝑁")
        print("6: Déchiffrement ElGamal de 𝑁")
        print("7: Quitter")
        choice = input("Votre choix : ")

        if choice == '1':
            names = input("Entrez les noms: ")
            target_zeros = int(input("Entrez le nombre de zéros cibles: "))
            find_string_with_sha256_ending_zeros(names, target_zeros)
            pass

        elif choice == '2':
            names = input("Entrez les noms: ")
            target_zeros = int(input("Entrez le nombre de zéros cibles: "))
            trials = int(input("Entrez le nombre d'essais: "))
            time_to_find_zeros(names, target_zeros, trials)
            pass

        elif choice == '3':
            mode = input("Voulez-vous entrer le texte manuellement (m) ou utiliser un fichier (f) ? (m/f): ")
            password = input("Entrez le mot de passe : ")  # Il est préférable de demander le mot de passe avant de choisir entre manuel et fichier
            if mode.lower() == 'm':
                input_text = input("Entrez le texte à chiffrer: ")
            elif mode.lower() == 'f':
                file_path = input("Entrez le chemin du fichier à chiffrer: ")
                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        input_text = file.read()
                except FileNotFoundError:
                    print("Fichier non trouvé. Assurez-vous que le chemin est correct.")
                    return
                except Exception as e:
                    print(f"Une erreur est survenue lors de la lecture du fichier: {e}")
                    return
            else:
                print("Choix non valide.")
                return
            # Appel à la fonction aes_ctr_encryption avec les variables correctement définies
            encrypted_text, iv = aes_ctr_encryption(input_text, password, salt, iv)  # La fonction doit retourner le texte chiffré et l'IV
            pass

        elif choice == '4':
            encrypted_text_hex = input("Entrez le texte chiffré (en hexadécimal) : ")
            password = input("Entrez le mot de passe : ")
            # Utilisez le même salt et le même iv que ceux utilisés pour le chiffrement
            salt = b'bonjour'
            iv = b'\x6a\x49\xd5\xbe\x1d\xec\xdf\xe4\x34\x1d\xdf\x5c\xfa\xb2\xe3\xd9'
            decrypted_text = aes_ctr_decryption(encrypted_text_hex, password, salt, iv)
            print(f"Texte déchiffré: {decrypted_text}")
            pass

        elif choice == '5':
            N = int(input("Entrez le nombre N à chiffrer : "))
            chiffrement_elgamal(N, p, g)
            pass

        elif choice == '6':
            x = int(input("Entrez la clé privée x : "))
            B = int(input("Entrez la partie B du message chiffré : "))
            C = int(input("Entrez la partie C du message chiffré : "))
            N_dechiffre = dechiffrement_elgamal(B, C, p, x)
            print(f"Message déc123hiffré : {N_dechiffre}")
            pass

        elif choice == '7':
            print("Au revoir!")
            break


        else:
                print("Choix non valide. Veuillez réessayer.")

                # Demander à l'utilisateur s'il souhaite effectuer une autre opération
        continuer = input("\nVoulez-vous effectuer une autre opération ? (oui/non) : ")
        if continuer.lower() != 'oui':
            print("Au revoir!")
            break

if __name__ == "__main__":
    main()


Choisissez une fonction:
1: find_string_with_sha256_ending_zeros
2: time_to_find_zeros
3: aes_ctr_encryption
4: Déchiffrement AES CTR
5: Chiffrement ElGamal de 𝑁
6: Déchiffrement ElGamal de 𝑁
7: Quitter


Entrez le nombre N à chiffrer : 354265170000756431  
Clé privée (x): 3225388602036651519028090
Clé publique (A): 4939783228691804329549996
B: 2581305696278121567553318
C: 4463318798022436691221647